O objetivo do código aqui é comparar os modelos no caso de um horizonte temporal $h = 1$

In [6]:
import utils
import numpy as np
import pandas as pd
import epftools as epf
from sktime.forecasting.compose import make_reduction
from sklearn.preprocessing import StandardScaler
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError, MeanAbsoluteError, MeanSquaredError
from sktime.forecasting.model_evaluation import evaluate
from sktime.split import ExpandingWindowSplitter, temporal_train_test_split
from sktime.forecasting.base import ForecastingHorizon
# Modelos
from sktime.forecasting.arima import AutoARIMA
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor


In [7]:
# Horizonte de previsões
# Deve ser a única coisa a mudar entre todos os arquivos h{i}.ipynb
FORECASTING_HORIZON = 1
TEST_SIZE = 79

In [8]:
# Importa dados sem lags
df = utils.read_and_change_index("dados/dados_transformados.csv")
df.head()

,ipca,ipca_15,selic,m1,m2,m3,m4,ibovespa,cambio,cambio_real,ind_varejo,ind_varejo_ampl,ind_industria,ibc_br,capacidade_industria,salario_minimo,energia_total,energia_residencial,energia_industrial,energia_comercial,energia_outros,oil_usa,oil_eu,pib,bens_capital,bens_int,bens_cons_d,bens_cons_nd,prod_veiculos,prod_caminhoes,prod_onibus,result_primario,el_nino,epu_br,epu_usa,ind_conf_cons,desemprego,rendimento,crb_cmdt,crb_food,crb_metal
month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01,0.76,0.68,-0.548387,-11559013.0,-22319905.0,-18028683.0,-17053471.0,-384.95,-0.0735,-0.363,-7.19247,-6.29474,-1.25943,-1.15,-1.4,0.0,-1585.77401,198.698508,-1501.06827,-200.182185,-83.222143,34.31,31.28,-2274.0,65.64871,1.56545,3.18406,-7.88372,150001,6245,212.0,2481.49,0.4,49.609174,-2.34359,8.9,14.696341,348.7366,7.70,18.70,15.10
2004-02,0.61,0.90,0.000000,1650795.0,11117714.0,1275340.0,-2131936.0,-96.42,0.0785,1.699,-0.78942,-1.02353,-3.21265,0.86,0.8,0.0,-470.18700,-351.442000,132.56500,-97.562000,-153.748000,34.68,30.86,-6479.7,65.72891,-2.12006,-8.67680,-5.52757,148766,7181,370.0,29.00,0.3,121.394093,1.00000,-0.8,15.073171,0.0000,10.11,14.74,24.27
2004-03,0.47,0.40,-0.112903,-1575433.0,-1282702.0,300227.0,-4199993.0,387.24,-0.0248,-2.777,1.53498,2.86590,15.02434,12.53,-0.7,0.0,776.15800,171.100000,466.99600,56.588000,81.474000,36.74,33.63,10259.0,82.95766,13.99826,25.26343,13.95086,185509,9212,402.0,2375.73,0.2,-166.416169,-5.00000,-10.1,16.078049,-10382.5684,4.04,2.71,10.95
2004-04,0.37,0.21,-0.270430,539382.0,-378329.0,-4736024.0,-2778970.0,-2535.03,0.0005,-1.562,0.13157,-0.81883,-5.50945,-4.62,-0.9,0.0,761.85300,208.669000,278.87800,243.212000,31.095000,36.75,33.59,-2418.6,76.27416,-4.78950,-8.60280,-5.81031,163375,8611,-33.0,-1911.68,0.2,21.276658,-4.00000,-5.3,16.454878,-339.4192,-0.45,6.35,-11.13
2004-05,0.51,0.54,-0.116667,1252881.0,13051576.0,26890443.0,11143701.0,-62.56,0.1944,4.213,1.09641,1.33059,4.81835,-1.33,0.8,20.0,-801.67600,-331.666000,-54.57900,-341.355000,-74.077000,40.28,37.57,1947.6,81.42104,5.58800,1.67710,3.80982,172380,9116,260.0,1488.90,0.2,14.892162,17.00000,16.2,15.324390,238.2590,-5.31,-15.02,8.03


In [9]:
# Benchmark
arima = AutoARIMA(sp=12)
# Cria dicionário com os outros modelos
forecasters = {
    'ridge': make_reduction(RidgeCV()),
    'lasso': make_reduction(LassoCV()),
    'random_forest': make_reduction(RandomForestRegressor(max_features="sqrt")),
    'lgbm': make_reduction(LGBMRegressor(max_depth=2)),
}

In [10]:
# Roda o ARIMA
# Serve apenas para achar o tamanho da 'initial_window' e para métricas de erro depois
y_train, y_test = temporal_train_test_split(df['ipca'], test_size=TEST_SIZE)
fh = ForecastingHorizon(FORECASTING_HORIZON, is_relative=True)
arima_eval_df = evaluate(
    arima,
    ExpandingWindowSplitter(fh, initial_window=y_train.size),
    df['ipca'],
    return_data=True
)
arima_pred = utils.extract_y_pred(arima_eval_df)
arima_pred

C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\asfra\AppData\Local\Packages

month
2017-03    0.282510
2017-04    0.401124
2017-05    0.309435
2017-06    0.281415
2017-07    0.056204
             ...   
2023-05    0.519261
2023-06    0.320844
2023-07   -0.002215
2023-08    0.248564
2023-09    0.299311
Freq: M, Name: ipca, Length: 79, dtype: float64

In [ ]:
# Não funciona pois modelo não foi "fitted"
#arima.get_fitted_params()

In [12]:
# Salva métricas de erro e previsões em dicionário
res = {}
preds = {}
# Salva métricas de erro do ARIMA
mae = MeanAbsoluteError()
mse = MeanSquaredError()
mape = MeanAbsolutePercentageError()
res['arima'] = [
    mae(y_test, arima_pred),
    mse(y_test, arima_pred),
    mape(y_test, arima_pred),
]
for model, forecaster in forecasters.items():
    mae = MeanAbsoluteError()
    mse = MeanSquaredError()
    mape = MeanAbsolutePercentageError()
    
    # Modelos lineares (precisam de dados normalizados)
    if model == "ridge" or model == "lasso":
        # Normaliza dados
        scaler = StandardScaler()
        normalized_data = scaler.fit_transform(df)
        normalized_df = pd.DataFrame(normalized_data, columns=df.columns, index=df.index)
        
        y_train, y_test, y_pred = utils.evaluate_pipeline(forecaster, normalized_df)
        # Desnormaliza previsões
        preds[model] = y_pred * np.sqrt(scaler.var_[0]) + scaler.mean_[0]

    else:
        y_train, y_test, y_pred = utils.evaluate_pipeline(forecaster, df)
        preds[model] = y_pred

    res[model] = [
        mae(y_test, y_pred),
        mse(y_test, y_pred),
        mape(y_test, y_pred)
    ]

C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.039e-03, tolerance: 8.957e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.159e-03, tolerance: 8.957e-03
  model = cd_fast.enet_coordinate_descent(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\loca

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002949 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 19353
[LightGBM] [Info] Number of data points in the train set: 148, number of used features: 400
[LightGBM] [Info] Start training from score 0.481959
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

In [13]:
res_df = pd.DataFrame(res, index=['MAE', 'MSE', 'MAPE'])
res_df

,arima,ridge,lasso,random_forest,lgbm
MAE,0.281731,1.158426,0.836165,0.298185,0.301838
MSE,0.143799,2.265470,1.232037,0.155537,0.157007
MAPE,1.590837,5.117048,3.168554,1.492521,1.279628


In [14]:
p_values = {}
for model, pred in preds.items():
    p_values[model] = epf.DM(y_test, arima_pred, pred)

In [15]:
pd.DataFrame(p_values, index=['DM'])

,ridge,lasso,random_forest,lgbm
DM,0.999644,0.335974,0.781475,0.859384


In [17]:
preds['arima'] = arima_pred
preds_df = pd.DataFrame(preds, index=y_test.index)
preds_df.head()

,ridge,lasso,random_forest,lgbm,arima
month,,,,,
2017-03,0.458606,0.413438,0.5037,0.506608,0.282510
2017-04,0.691161,0.425752,0.3948,0.402017,0.401124
2017-05,0.060418,0.123628,0.3093,0.094456,0.309435
2017-06,-0.115428,0.198299,0.2641,0.075895,0.281415
2017-07,-0.141429,0.042775,0.2642,0.117071,0.056204


In [19]:
preds_df.to_csv("previsões/h1.csv", sep=';', decimal=',')

In [21]:
forecasters['random_forest'].get_fitted_params()

NotFittedError: estimator of type RecursiveTabularRegressionForecaster has not been fitted yet, please call fit on data before get_fitted_params

In [22]:
# TESTE
y = df['ipca']
X = df.drop(columns=['ipca'])
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=TEST_SIZE)
fh = ForecastingHorizon(FORECASTING_HORIZON, is_relative=True)
cv = ExpandingWindowSplitter(fh=fh, initial_window=y_train.size)
forecasters['random_forest'].fit(y_train)
rf_y_pred = forecasters['random_forest'].update_predict(y, cv=cv)

C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\forecasting\base\_base.py:1919: UserWarning: NotImplementedWarning: RecursiveTabularRegressionForecaster does not have a custom `update` method implemented. RecursiveTabularRegressionForecaster will be refit each time `update` is called with update_params=True. To refit less often, use the wrappers in the forecasting.stream module, e.g., UpdateEvery.
  warn(
C:\Users\asfra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sktime\forecasting\base\_base.py:1919: UserWarning: NotImplementedWarning: RecursiveTabularRegressionForecaster does not have a custom `update` method implemented. RecursiveTabularRegressionForecaster will be refit each time `update` is called with update_params=True. To refit less often, use the wrappers in the forecasting.stream module, e.g., UpdateE

In [23]:
rf_y_pred

2017-03    0.4155
2017-04    0.4072
2017-05    0.3440
2017-06    0.3705
2017-07    0.3470
            ...  
2023-05    0.5704
2023-06    0.3387
2023-07    0.1303
2023-08    0.3237
2023-09    0.3449
Freq: M, Name: ipca, Length: 79, dtype: float64